In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import math
# Any results you write to the current directory are saved as output.

/home/tannmay/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/tannmay/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [10]:
x_train = pd.read_csv("Training_dataset_Original.csv")
test = pd.read_csv("Leaderboard_dataset.csv")
val_X = pd.read_csv("Evaluation_dataset.csv")
dic_df = pd.read_csv("Data_Dictionary.csv")

/home/tannmay/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
test.shape 

(25000, 48)

In [12]:
ID = test["application_key"]
test = test.drop("application_key", axis=1)

In [13]:
test.shape

(25000, 47)

In [14]:
# import matplotlib.pyplot as plt
# X_train[X_train.dtypes[(X_train.dtypes=="float64")|(X_train.dtypes=="int64")]
#                         .index.values].hist(figsize=[11,11])

In [15]:
# len(train_df)
# len(val_df)
# len(sub_test)
# display(dic_df)
# train_df.head()
# val_df.head()

In [17]:
cols = list(x_train.columns.values)


In [20]:
y_train = x_train["default_ind"]
y_train.head()
x_train = x_train.drop('default_ind', axis=1)
x_train = x_train.drop('application_key', axis=1)
display(x_train.head())
display(y_train.head())

,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,mvar10,...,mvar38,mvar39,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45,mvar46,mvar47
0,1696,1.6541,0.000,0.0,0.0,0,6015,322,40369,18414,...,4,1,73.78,82.547,0.08696,10,0.63899,na,0,C
1,1846,0.8095,0.000,0.0,0.0,102,7532,3171,18234,13664,...,2,0,99.129,missing,0,13,0.63836,na,na,L
2,1745,0.4001,0.000,0.0,0.0,missing,2536,missing,missing,2536,...,1,0,missing,29.29,0,1,1.00000,na,0,C
3,1739,0.2193,0.000,0.0,0.0,1982,26440,4955,20316,37013,...,2,0,96.272,missing,0.15385,3,0.53241,0,0,L
4,1787,0.0118,0.225,0.0,0.0,5451,5494,5494,7987,4696,...,2,0,115.019,missing,0,1,0.92665,na,na,L


0    0
1    1
2    1
3    0
4    0
Name: default_ind, dtype: int64

In [21]:
# Missing data handling methods
"""
 important features by hand picking
 mvar1 mvar2 mvar3 mvar14 mvar42

 mvar3 - mvar4 - mvar5 -> linearly dependent as severity is a function of amount, time since default and number of defaults
 mvar6-mvar7-mvar8 -> linearly dependent
 mvar16-mvar17-mvar18-mvar19-mvar20 -> Linearly Dependent
 mvar21-mvar22-mvar23-mvar24 -> Linearly Dependet
 mvar25-mvar26-mvar27-mvar28-mvar29-mvar30-mvar31-mvar32 -> Linearly Dependent
 mvar34-mvar35-mvar36-mvar37-mvar38-mvar39 -> Linearly Dependent
 mvar14-mvar9 -> linearly dependent

 Handling
 mvar2 and dependent terms -> 0
 mvar3 and dependent terms -> 0
 mvar6 and dependent terms -> missing -> 0
 mvar9 and dependent terms -> missing -> mean
 mvar11 and dependent terms -> mean or 0
 mvar9 and dependent terms -> mean
 mvar15 and dependent terms -> mean
 mvar21 -> mean
 mvar16 and dependent terms -> 0
 mvar21 and dependent terms -> mean
 mvar25 and dependent terms -> mean
 mvar33 -> mean
 mvar34 and dependent terms -> mean_floor
 mvar42 -> mean
 mvar43 -> mean
 mvar44 -> mean
 mvar45 and mvar46 -> 0
"""

'\n important features by hand picking\n mvar1 mvar2 mvar3 mvar14 mvar42\n\n mvar3 - mvar4 - mvar5 -> linearly dependent as severity is a function of amount, time since default and number of defaults\n mvar6-mvar7-mvar8 -> linearly dependent\n mvar16-mvar17-mvar18-mvar19-mvar20 -> Linearly Dependent\n mvar21-mvar22-mvar23-mvar24 -> Linearly Dependet\n mvar25-mvar26-mvar27-mvar28-mvar29-mvar30-mvar31-mvar32 -> Linearly Dependent\n mvar34-mvar35-mvar36-mvar37-mvar38-mvar39 -> Linearly Dependent\n mvar14-mvar9 -> linearly dependent\n\n Handling\n mvar2 and dependent terms -> 0\n mvar3 and dependent terms -> 0\n mvar6 and dependent terms -> missing -> 0\n mvar9 and dependent terms -> missing -> mean\n mvar11 and dependent terms -> mean or 0\n mvar9 and dependent terms -> mean\n mvar15 and dependent terms -> mean\n mvar21 -> mean\n mvar16 and dependent terms -> 0\n mvar21 and dependent terms -> mean\n mvar25 and dependent terms -> mean\n mvar33 -> mean\n mvar34 and dependent terms -> mean_f

In [22]:
test.shape

(25000, 47)

In [23]:
def impute_nan(train):
    a = []
    for i in train:
        if math.isnan(i):
            continue
        else:
            a.append(float(i))

    np.mean(a)

    b = []
    for i in train:
        if math.isnan(i):
            b.append(np.median(a))
        else:
            b.append(float(i))
            
            
    return b

In [24]:
def impute_na(train):
    a = []
    for i in train:
        if i == 'na':
            continue
        else:
            a.append(float(i))

    np.median(a)

    b = []
    for i in train:
        if i == 'na':
            b.append(np.median(a))
        else:
            b.append(float(i))
            
            
    return b

In [25]:
def impute_missing(train):
    a = []
    for i in train:
        if i == 'missing':
            continue
        else:
            a.append(float(i))

    np.median(a)

    b = []
    for i in train:
        if i == 'missing':
            b.append(np.median(a))
        else:
            b.append(float(i))
            
            
    return b

In [26]:
def impute_missing_0(train):
    b = []
    for i in train:
        if i == 'missing':
            b.append(0)
        else:
            b.append(float(i))
            
            
    return b

In [27]:
def impute_na_0(train):
    b = []
    for i in train:
        if i == 'na':
            b.append(0)
        else:
            b.append(float(i))
            
            
    return b

In [28]:
def impute_nan_0(train):
    b = []
    for i in train:
        if math.isnan(i):
            b.append(0)
        else:
            b.append(float(i))
            
            
    return b

In [29]:
x_train.mvar1 = impute_na(x_train.mvar1)
x_train.mvar16 = impute_na_0(x_train.mvar16)
x_train.mvar17 = impute_na_0(x_train.mvar17)
x_train.mvar18 = impute_na_0(x_train.mvar18)
# x_train.mvar19 = impute_na(x_train.mvar19)
# x_train.mvar20 = impute_na(x_train.mvar20)
# x_train.mvar34 = impute_na(x_train.mvar34)
# x_train.mvar35 = impute_na(x_train.mvar35)
x_train.mvar36 = impute_na(x_train.mvar36)
x_train.mvar37 = impute_na(x_train.mvar37)
x_train.mvar38 = impute_na(x_train.mvar38)
x_train.mvar39 = impute_na_0(x_train.mvar39)
# x_train.mvar43 = impute_na(x_train.mvar43)
x_train.mvar45 = impute_na_0(x_train.mvar45)
x_train.mvar46 = impute_na_0(x_train.mvar46)

x_train.mvar2 = impute_nan(x_train.mvar2)
x_train.mvar3 = impute_nan(x_train.mvar3)
x_train.mvar4 = impute_nan(x_train.mvar4)
x_train.mvar5 = impute_nan(x_train.mvar5)
x_train.mvar21 = impute_nan(x_train.mvar21)
x_train.mvar22 = impute_nan(x_train.mvar22)
x_train.mvar23 = impute_nan(x_train.mvar23)
x_train.mvar24 = impute_nan_0(x_train.mvar24)
x_train.mvar33 = impute_nan(x_train.mvar33)
x_train.mvar44 = impute_nan_0(x_train.mvar44)

x_train.mvar6 = impute_missing(x_train.mvar6)
x_train.mvar7 = impute_missing(x_train.mvar7)
x_train.mvar8 = impute_missing(x_train.mvar8)
x_train.mvar9 = impute_missing(x_train.mvar9)
x_train.mvar10 = impute_missing(x_train.mvar10)
x_train.mvar11 = impute_missing_0(x_train.mvar11)
x_train.mvar12 = impute_missing(x_train.mvar12)
x_train.mvar13 = impute_missing(x_train.mvar13)
x_train.mvar15 = impute_missing(x_train.mvar15)
x_train.mvar25 = impute_missing_0(x_train.mvar25)
x_train.mvar26 = impute_missing_0(x_train.mvar26)
x_train.mvar27 = impute_missing_0(x_train.mvar27)
x_train.mvar28 = impute_missing_0(x_train.mvar28)
x_train.mvar29 = impute_missing_0(x_train.mvar29)
x_train.mvar30 = impute_missing(x_train.mvar30)
x_train.mvar31 = impute_missing(x_train.mvar31)
# x_train.mvar32 = impute_missing(x_train.mvar32)

x_train.mvar40 = impute_missing(x_train.mvar40)
x_train.mvar41 = impute_missing(x_train.mvar41)
x_train.mvar42 = impute_missing(x_train.mvar42)


In [30]:
d0 = pd.DataFrame(data=x_train)
d0.to_csv("x_train_imputed.csv", index=False)

In [33]:
val_X = val_X.drop("application_key", axis=1)

In [ ]:
val_X.mvar1 = impute_na(val_X.mvar1)
val_X.mvar16 = impute_na_0(val_X.mvar16)
val_X.mvar17 = impute_na_0(val_X.mvar17)
val_X.mvar18 = impute_na_0(val_X.mvar18)
# val_X.mvar19 = impute_na(val_X.mvar19)
# val_X.mvar20 = impute_na(val_X.mvar20)
# val_X.mvar34 = impute_na(val_X.mvar34)
# val_X.mvar35 = impute_na(val_X.mvar35)
val_X.mvar36 = impute_na(val_X.mvar36)
val_X.mvar37 = impute_na(val_X.mvar37)
val_X.mvar38 = impute_na(val_X.mvar38)
val_X.mvar39 = impute_na_0(val_X.mvar39)
# val_X.mvar43 = impute_na(val_X.mvar43)
val_X.mvar45 = impute_na_0(val_X.mvar45)
val_X.mvar46 = impute_na_0(val_X.mvar46)

val_X.mvar2 = impute_nan(val_X.mvar2)
val_X.mvar3 = impute_nan(val_X.mvar3)
val_X.mvar4 = impute_nan(val_X.mvar4)
val_X.mvar5 = impute_nan(val_X.mvar5)
val_X.mvar21 = impute_nan(val_X.mvar21)
val_X.mvar22 = impute_nan(val_X.mvar22)
val_X.mvar23 = impute_nan(val_X.mvar23)
val_X.mvar24 = impute_nan_0(val_X.mvar24)
val_X.mvar33 = impute_nan(val_X.mvar33)
val_X.mvar44 = impute_nan_0(val_X.mvar44)

val_X.mvar6 = impute_missing(val_X.mvar6)
val_X.mvar7 = impute_missing(val_X.mvar7)
val_X.mvar8 = impute_missing(val_X.mvar8)
val_X.mvar9 = impute_missing(val_X.mvar9)
val_X.mvar10 = impute_missing(val_X.mvar10)
val_X.mvar11 = impute_missing_0(val_X.mvar11)
val_X.mvar12 = impute_missing(val_X.mvar12)
val_X.mvar13 = impute_missing(val_X.mvar13)
val_X.mvar15 = impute_missing(val_X.mvar15)
val_X.mvar25 = impute_missing_0(val_X.mvar25)
val_X.mvar26 = impute_missing_0(val_X.mvar26)
val_X.mvar27 = impute_missing_0(val_X.mvar27)
val_X.mvar28 = impute_missing_0(val_X.mvar28)
val_X.mvar29 = impute_missing_0(val_X.mvar29)
val_X.mvar30 = impute_missing(val_X.mvar30)
val_X.mvar31 = impute_missing(val_X.mvar31)
# val_X.mvar32 = impute_missing(val_X.mvar32)

val_X.mvar40 = impute_missing(val_X.mvar40)
val_X.mvar41 = impute_missing(val_X.mvar41)
val_X.mvar42 = impute_missing(val_X.mvar42)

In [ ]:
d1 = pd.DataFrame(data=val_X)
d1.to_csv("val_X_imputed.csv", index=False)

In [ ]:
test.mvar1 = impute_na(test.mvar1)
test.mvar16 = impute_na_0(test.mvar16)
test.mvar17 = impute_na_0(test.mvar17)
test.mvar18 = impute_na_0(test.mvar18)
# test.mvar19 = impute_na(test.mvar19)
# test.mvar20 = impute_na(test.mvar20)
# test.mvar34 = impute_na(test.mvar34)
# test.mvar35 = impute_na(test.mvar35)
test.mvar36 = impute_na(test.mvar36)
test.mvar37 = impute_na(test.mvar37)
test.mvar38 = impute_na(test.mvar38)
test.mvar39 = impute_na_0(test.mvar39)
# test.mvar43 = impute_na(test.mvar43)
test.mvar45 = impute_na_0(test.mvar45)
test.mvar46 = impute_na_0(test.mvar46)

test.mvar2 = impute_nan(test.mvar2)
test.mvar3 = impute_nan(test.mvar3)
test.mvar4 = impute_nan(test.mvar4)
test.mvar5 = impute_nan(test.mvar5)
test.mvar21 = impute_nan(test.mvar21)
test.mvar22 = impute_nan(test.mvar22)
test.mvar23 = impute_nan(test.mvar23)
test.mvar24 = impute_nan_0(test.mvar24)
test.mvar33 = impute_nan(test.mvar33)
test.mvar44 = impute_nan_0(test.mvar44)

test.mvar6 = impute_missing(test.mvar6)
test.mvar7 = impute_missing(test.mvar7)
test.mvar8 = impute_missing(test.mvar8)
test.mvar9 = impute_missing(test.mvar9)
test.mvar10 = impute_missing(test.mvar10)
test.mvar11 = impute_missing_0(test.mvar11)
test.mvar12 = impute_missing(test.mvar12)
test.mvar13 = impute_missing(test.mvar13)
test.mvar15 = impute_missing(test.mvar15)
test.mvar25 = impute_missing_0(test.mvar25)
test.mvar26 = impute_missing_0(test.mvar26)
test.mvar27 = impute_missing_0(test.mvar27)
test.mvar28 = impute_missing_0(test.mvar28)
test.mvar29 = impute_missing_0(test.mvar29)
test.mvar30 = impute_missing(test.mvar30)
test.mvar31 = impute_missing(test.mvar31)
# test.mvar32 = impute_missing(test.mvar32)

test.mvar40 = impute_missing(test.mvar40)
test.mvar41 = impute_missing(test.mvar41)
test.mvar42 = impute_missing(test.mvar42)

In [ ]:
d = pd.DataFrame(data=y_train)
d.to_csv("y_train_imputed.csv", index=False)

In [344]:


# Drop 19, 20, 35, 34, 32 and 43




# X = X_train[:50000]
# y = y_train[:50000]
# val_X = X_train[50000:70000]
# val_y = y_train[50000:70000]
# test_x = X_train[70000:]
# test_y = y_train[70000:]

In [345]:
# from sklearn.feature_selection import SelectKBest
# from sklearn.feature_selection import chi2
# # feature extraction
# test = SelectKBest(score_func=chi2, k=12)
# fit = test.fit(val_X, val_y)
# # summarize scores
# np.set_printoptions(precision=3)
# print(fit.scores_)
# features = fit.transform(X)
# # summarize selected features
# print(features[0:10,:])

In [346]:
# a = [1.579e+04, 2.668e+03, 1.420e+04, 2.422e+03, 4.398e+03, 4.909e+06, 4.155e+07,
#  1.472e+07, 7.266e+07, 1.016e+08, 2.479e+06, 1.546e+06, 3.889e+07, 1.979e+08,
#  5.342e+07, 1.262e+03, 1.482e+03, 1.475e+03, 1.357e+03, 1.243e+03, 2.489e+04,
#  2.661e+06, 1.773e+06, 3.800e+04, 7.827e+05, 2.086e+06, 1.798e+06, 1.936e+07,
#  1.708e+06, 7.711e+04, 3.581e+04, 5.562e+05, 3.228e+03, 1.003e+03, 1.165e+03,
#  2.175e+03, 4.518e+03, 1.881e+03, 7.315e+02, 2.664e+02, 3.403e+02, 8.375e+02,
#  3.037e+03, 6.543e+00, 2.754e+01, 4.569e+02, 6.566e+02]


In [347]:
test.shape

(25000, 47)

In [348]:
# for i in range(12):
#     print(a.index(b[i])+1)

In [349]:
# b = [197900000.0,
#  101600000.0,
#  72660000.0,
#  53420000.0,
#  41550000.0,
#  38890000.0,
#  19360000.0,
#  14720000.0,
#  4909000.0,
#  2661000.0,
#  2479000.0,
#  2086000.0]

In [350]:
# # Feature Extraction with PCA
# from sklearn.decomposition import PCA
# # feature extraction
# pca = PCA(n_components=12)
# fit = pca.fit(X)
# # summarize components
# print(("Explained Variance: %s") % fit.explained_variance_ratio_)
# print(fit.components_)

In [351]:
# X = np.dot(X, fit.components_.T)

In [352]:
test.shape

(25000, 47)

In [353]:
# test_x = np.dot(test_x, fit.components_.T)

In [354]:
# univariate selection
# column = ['mvar14', 'mvar10', 'mvar9', 'mvar15', 'mvar7', 'mvar13', 'mvar28', 'mvar8', 'mvar6', 'mvar22', 'mvar11', 'mvar26']
# X = X[column]
# test_x = test_x[column]
# test = test[column]

In [355]:
# test = np.dot(test, fit.components_.T)

In [356]:
# # L = 0
# from sklearn.preprocessing import LabelEncoder
# x_train.mvar47 = LabelEncoder().fit_transform(x_train.mvar47) 
# val_x.mvar47 = LabelEncoder().fit_transform(val_x.mvar47)
# sub_test.mvar47 = LabelEncoder().fit_transform(sub_test.mvar47)




In [357]:
# # Train, Validation and Test Set Declaration
# # sub_test is the performance set

# X = x_train[:70000]
# y = y_train[:70000]

# test_X = x_train[70000:]
# test_y = y_train[70000:]

# val_x.head()
# sub_test.head()

In [358]:
# from sklearn.linear_model import LogisticRegression
from sklearn import metrics
# from sklearn.ensemble import RandomForestClassifier


# clf = LogisticRegression(random_state=0, solver='lbfgs',
#                          multi_class='ovr').fit(X, y)

In [359]:
# y_predicted = clf.predict(test_X)

In [361]:
# np.mean(y_predicted == test_y)
test.shape

(25000, 47)

In [360]:
column = ['mvar2', 'mvar33', 'mvar1', 'mvar29', 'mvar7', 'mvar25', 'mvar13', 'mvar44', 'mvar14', 'mvar10', 'mvar42', 'mvar15',
          'mvar9', 'mvar11', 'mvar12', 'mvar24', 'mvar3', 'mvar30', 'mvar26', 'mvar32', 'mvar6', 'mvar21', 'mvar27', 'mvar8', 'mvar22', 'mvar36']


In [362]:
X = X[column]
val_X = val_X[column]
test_x = test_x[column]
test = test[column]

In [312]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X, y)

predicted_RandomForest = classifier.predict(test_x)
metrics.accuracy_score(test_y, predicted_RandomForest, normalize=True, sample_weight=None)

/home/tannmay/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


0.7382

In [313]:
#        num_leaves: 16
#      n_estimators: 246
#         max_depth: 50
#           max_bin: 128
#  bagging_fraction: 1.0
#  feature_fraction: 1.0
#     boosting_type: dart
#     learning_rate: 0.18456732832847952

In [452]:
def get_lgb_model():
    lgb_model = lgb.LGBMClassifier(objective='binary',num_leaves=43,
                      learning_rate=0.05,boosting_type='gbdt', n_estimators=50, max_depth=10,
                      metric='multi_logloss',is_training_metric=True,
                      max_bin = 1024, bagging_fraction = 1.0,verbose=-1,
                      bagging_freq = 100, feature_fraction = 1.0) 
    return lgb_model

import lightgbm as lgb

lgb_model = get_lgb_model()
lgb_model.fit(X, y)
target_hat = lgb_model.predict(test_x)

/home/tannmay/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tannmay/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tannmay/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [453]:
metrics.accuracy_score(test_y, target_hat, normalize=True, sample_weight=None)

0.7702

In [ ]:
metrics.accuracy_score(test_y, target_hat, normalize=True, sample_weight=None)

In [316]:
import xgboost

In [320]:
xgb_params = {
    "objective" : "multi:softprob",
    "num_class" : 2,
    "tree_method" : "auto",
    "eval_metric" : "mlogloss",
    "nthread": 4,
    "seed" : 0,
    'silent': 1,

    "eta":0.250,  # default 0.3
    "max_depth" : 15, # default 6
    "subsample" : 0.957, # default 1
    "colsample_bytree" : 0.4390, # default 1
    "gamma": 0.5
}


In [321]:
model = xgboost.XGBClassifier(**xgb_params)
model.fit(X, y)
y_pred = model.predict(test_x)
metrics.accuracy_score(test_y, y_pred, normalize=True, sample_weight=None)

/home/tannmay/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tannmay/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tannmay/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.7452

In [322]:
y_prediction = model.predict(test)


/home/tannmay/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [233]:
# ID = test["application_key"]
# # sub_test = sub_test.drop("application_key", axis=1)

In [234]:
d = {'Id': ID, 'Target': y_prediction}
submission = pd.DataFrame(data=d)
submission.to_csv('submission.csv', header = False, index=False)
